In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets/organizations/zalando-research/fashionmnist/t10k-labels-idx1-ubyte
/kaggle/input/datasets/organizations/zalando-research/fashionmnist/t10k-images-idx3-ubyte
/kaggle/input/datasets/organizations/zalando-research/fashionmnist/fashion-mnist_test.csv
/kaggle/input/datasets/organizations/zalando-research/fashionmnist/fashion-mnist_train.csv
/kaggle/input/datasets/organizations/zalando-research/fashionmnist/train-labels-idx1-ubyte
/kaggle/input/datasets/organizations/zalando-research/fashionmnist/train-images-idx3-ubyte


In [13]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("zalando-research/fashionmnist")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/datasets/organizations/zalando-research/fashionmnist


In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, regularizers

# 1. LOAD DATA
train_df = pd.read_csv('/kaggle/input/datasets/organizations/zalando-research/fashionmnist/fashion-mnist_train.csv')
test_df = pd.read_csv('/kaggle/input/datasets/organizations/zalando-research/fashionmnist/fashion-mnist_test.csv')

# 2. PREPROCESS DATA
# Extract labels (y) and pixel values (X)
y_train_raw = train_df.iloc[:, 0].values
x_train_raw = train_df.iloc[:, 1:].values

y_test_raw = test_df.iloc[:, 0].values
x_test_raw = test_df.iloc[:, 1:].values

# Normalize pixel values to [0, 1] as per Unit I guidelines
x_train = x_train_raw.astype("float32") / 255.0
x_test = x_test_raw.astype("float32") / 255.0

# One-hot encode labels (Unit II Requirement)
y_train = to_categorical(y_train_raw, 10)
y_test = to_categorical(y_test_raw, 10)

# 3. BUILD MODEL WITH L2 REGULARIZATION
def build_l2_model():
    model = models.Sequential([
        layers.Dense(128, activation='relu', input_shape=(784,),
                     kernel_regularizer=regularizers.l2(0.01)),
        layers.Dense(64, activation='relu',
                     kernel_regularizer=regularizers.l2(0.01)),
        layers.Dense(10, activation='softmax')
    ])
    return model

model_l2 = build_l2_model()

# 4. COMPILE AND TRAIN
model_l2.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history = model_l2.fit(
    x_train, y_train,
    epochs=10,
    validation_split=0.2, # Monitoring for overfitting
    batch_size=32
)

# 5. EVALUATE
test_loss, test_acc = model_l2.evaluate(x_test, y_test)
print(f"\n--- Fashion-MNIST Results ---")
print(f"Test accuracy: {test_acc:.4f}")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.7345 - loss: 1.6552 - val_accuracy: 0.7940 - val_loss: 0.7770
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8024 - loss: 0.7442 - val_accuracy: 0.8200 - val_loss: 0.6956
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8172 - loss: 0.6804 - val_accuracy: 0.8227 - val_loss: 0.6636
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8211 - loss: 0.6384 - val_accuracy: 0.8050 - val_loss: 0.6824
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8295 - loss: 0.6154 - val_accuracy: 0.8311 - val_loss: 0.6078
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8293 - loss: 0.6050 - val_accuracy: 0.8360 - val_loss: 0.5878
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8305 - loss: 0.5886 - val_accuracy: 0.8332 - val_loss: 0.5958
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8365 - loss: 0.5725 - 

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import fashion_mnist # Changed from mnist
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 1. Load Fashion MNIST
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# 2. Preprocess
# Normalize to [0, 1]
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Reshape for ImageDataGenerator (Height, Width, Channels)
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# One-hot encoding for the 10 clothing classes
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 3. Data Augmentation (Modified for Fashion Data)
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,  # Set to True for clothes! 
    vertical_flip=False    # Keep False (we don't wear shirts upside down)
)



datagen.fit(x_train)

# 4. Build MLP Model with L2 Regularization
def build_l2_model():
    model = models.Sequential([
        layers.Flatten(input_shape=(28, 28, 1)),
        # Adding more neurons (128) because Fashion-MNIST is more complex than digits
        layers.Dense(128, activation='relu',
                     kernel_regularizer=regularizers.l2(0.001)), 
        layers.Dropout(0.2), # Adding Dropout as per Week 5 Syllabus
        layers.Dense(64, activation='relu',
                     kernel_regularizer=regularizers.l2(0.001)),
        layers.Dense(10, activation='softmax')
    ])
    return model

model_l2 = build_l2_model()

# 5. Compile
model_l2.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 6. Train using the generator
history = model_l2.fit(
    datagen.flow(x_train, y_train, batch_size=32),
    epochs=15, # Increased epochs as augmentation makes learning slower but more robust
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // 32
)

# 7. Evaluate
test_loss, test_acc = model_l2.evaluate(x_test, y_test)
print(f"\nFashion-MNIST Test accuracy: {test_acc:.4f}")

Epoch 1/15


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  25/1875 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.2370 - loss: 2.4171

In [ ]:
import tensorflow as tf

# Check if GPU is available
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('GPU device not found. Check settings!')
else:
    print('Found GPU at: {}'.format(device_name))


Early Stopping

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

# 1. LOAD DAta
train_df = pd.read_csv('/kaggle/input/datasets/organizations/zalando-research/fashionmnist/fashion-mnist_train.csv')
test_df = pd.read_csv('/kaggle/input/datasets/organizations/zalando-research/fashionmnist/fashion-mnist_test.csv')



# 2. PREPROCESS
y_train = to_categorical(train_df.iloc[:, 0].values, 10)
x_train = train_df.iloc[:, 1:].values.astype("float32") / 255.0

y_test = to_categorical(test_df.iloc[:, 0].values, 10)
x_test = test_df.iloc[:, 1:].values.astype("float32") / 255.0

# 3. BUILD MODEL
def build_model():
    model = models.Sequential([
        # Fashion-MNIST is complex; using 128 neurons for better representation
        layers.Dense(128, activation='relu', input_shape=(784,)),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model

model = build_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 4. EARLY STOPPING (Unit II Regularization)
# This prevents overfitting by stopping training when validation loss stops decreasing
earlystop = EarlyStopping(
    monitor='val_loss',       # Monitoring loss is usually better for early stopping
    patience=5,               # Give it 5 epochs to improve before quitting
    restore_best_weights=True 
)



# 5. TRAIN
history = model.fit(
    x_train, y_train,
    epochs=50,                # Set high; EarlyStopping will handle the cut-off
    batch_size=256,
    validation_split=0.2,
    callbacks=[earlystop],
    verbose=1
)

# 6. EVALUATE & VISUALIZE
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"\n✅ Fashion-MNIST Test accuracy: {test_acc:.4f}")

# Plotting Loss to visualize Early Stopping for your lab report
plt.figure(figsize=(10, 4))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Effect of Early Stopping on Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Drop out

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models

# 1. LOAD FASHION-MNIST (Using local Kaggle paths)
train_df = pd.read_csv('/kaggle/input/datasets/organizations/zalando-research/fashionmnist/fashion-mnist_train.csv')
test_df = pd.read_csv('/kaggle/input/datasets/organizations/zalando-research/fashionmnist/fashion-mnist_test.csv')


# 2. PREPROCESS
y_train = to_categorical(train_df.iloc[:, 0].values, 10)
x_train = train_df.iloc[:, 1:].values.astype("float32") / 255.0

y_test = to_categorical(test_df.iloc[:, 0].values, 10)
x_test = test_df.iloc[:, 1:].values.astype("float32") / 255.0

# 3. BUILD MODEL WITH DROPOUT
# Note: Since you have a GPU, we can increase the neurons to 256 
# to give the model more 'learning capacity' before Dropout trims it down.
def build_dropout_model():
    model = models.Sequential([
        layers.Dense(256, activation='relu', input_shape=(784,)),
        layers.Dropout(0.3),  # 🔹 Randomly sets 30% of inputs to 0 during training
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),  # 🔹 Helps prevent neurons from co-depending too much
        layers.Dense(10, activation='softmax')
    ])
    return model

# [Image of dropout regularization in neural networks showing active and inactive neurons]

# 4. COMPILE (Using Adam as per your syllabus)
model = build_dropout_model()
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 5. TRAIN 
# With the GPU Tesla T4, we can handle a larger batch size (512) for faster training
history = model.fit(
    x_train, y_train,
    epochs=25, 
    batch_size=512,
    validation_split=0.2,
    verbose=1
)

# 6. EVALUATE
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"\n Fashion-MNIST with Dropout Test accuracy: {test_acc:.4f}")

Adding Noise

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models

# 1. LOAD DAAta
train_df = pd.read_csv('/kaggle/input/datasets/organizations/zalando-research/fashionmnist/fashion-mnist_train.csv')
test_df = pd.read_csv('/kaggle/input/datasets/organizations/zalando-research/fashionmnist/fashion-mnist_test.csv')

# 2. PREPROCESS CLEAN DATA
y_train = to_categorical(train_df.iloc[:, 0].values, 10)
x_train = train_df.iloc[:, 1:].values.astype("float32") / 255.0

y_test = to_categorical(test_df.iloc[:, 0].values, 10)
x_test = test_df.iloc[:, 1:].values.astype("float32") / 255.0

# ===============================
# 🔹 3. ADD NOISE TO INPUTS (Unit II Regularization)
# ===============================
# We add Gaussian noise to the images
noise_factor = 0.25 
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape)

# Clip back to [0, 1] range
x_train_noisy = np.clip(x_train_noisy, 0., 1.)

# 

# ===============================
# 🔹 4. ADD NOISE TO OUTPUTS (Label Noise)
# We intentionally 'lie' to the model for 10% of labels to test robustness
# ===============================
noise_ratio = 0.1
num_samples = int(noise_ratio * y_train.shape[0])
random_indices = np.random.choice(y_train.shape[0], num_samples, replace=False)

for idx in random_indices:
    # Generate a random label from 0 to 9
    random_label = np.random.randint(0, 10)
    y_train[idx] = to_categorical(random_label, 10)

# ===============================
# 5. BUILD MODEL
# ===============================
def build_model():
    model = models.Sequential([
        # Since input is noisy, a slightly larger layer helps capture signal
        layers.Dense(128, activation='relu', input_shape=(784,)),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    return model

model = build_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 6. TRAIN (On Noisy Data, validating on Clean Data)
# Using GPU power with batch_size 512
history = model.fit(
    x_train_noisy, 
    y_train, 
    epochs=25, 
    batch_size=512, 
    validation_data=(x_test, y_test), # Check if model generalizes to clean data
    verbose=1
)

# 7. EVALUATE
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"\n Robustness Test Results (Fashion-MNIST)")
print(f"Test accuracy on Clean Data: {test_acc:.4f}")

Parameter sharing

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model

# 1. LOAD DATA
train_df = pd.read_csv('/kaggle/input/datasets/organizations/zalando-research/fashionmnist/fashion-mnist_train.csv')
test_df = pd.read_csv('/kaggle/input/datasets/organizations/zalando-research/fashionmnist/fashion-mnist_test.csv')

# 2. PREPROCESS
y_train = to_categorical(train_df.iloc[:, 0].values, 10)
x_train = train_df.iloc[:, 1:].values.astype("float32") / 255.0

y_test = to_categorical(test_df.iloc[:, 0].values, 10)
x_test = test_df.iloc[:, 1:].values.astype("float32") / 255.0

# 3. BUILD SHARED ARCHITECTURE (Functional API)
# Parameter Sharing: The 'shared_dense' layer has ONE set of weights (W, b)
shared_dense = Dense(128, activation='relu', name="shared_feature_extractor")

# Define two input branches
input_a = Input(shape=(784,), name="Input_Branch_A")
input_b = Input(shape=(784,), name="Input_Branch_B")

# Both inputs use the SAME layer instance (Weights are tied/shared)
feat_a = shared_dense(input_a)
feat_b = shared_dense(input_b)

# Merge the shared representations
merged = Concatenate()([feat_a, feat_b])

# Final Classification Head
hidden = Dense(64, activation='relu')(merged)
output = Dense(10, activation='softmax')(merged)

# Construct Model
model = Model(inputs=[input_a, input_b], outputs=output)


# 4. COMPILE & TRAIN
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Train (Using GPU: Tesla T4)
# In this demo, we feed the same image to both branches. 
# In a real project, you might feed an image and its augmented version!
history = model.fit([x_train, x_train], y_train,
                    epochs=15,
                    batch_size=512,
                    validation_split=0.2)

# 5. EVALUATE
test_loss, test_acc = model.evaluate([x_test, x_test], y_test)
print(f"\nFashion-MNIST Parameter Sharing Accuracy: {test_acc:.4f}")